### 0. Импорт библиотек

In [37]:
!pip install torch

In [1]:
# база
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.metrics import f1_score

2025-09-26 11:48:38.376458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758887318.551311      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758887318.605058      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### 1. Импорт датасета

In [2]:
data_path = '/kaggle/input/kinopoisk-review/'

In [3]:
df = pd.read_json(f'{data_path}/kinopoisk.jsonl', lines=True)
df.head()

,part,movie_name,review_id,author,date,title,grade3,grade10,content
0,top250,Блеф (1976),17144,Come Back,2011-09-24,Плакали наши денежки ©,Good,10.0,"\n""Блеф» — одна из моих самых любимых комедий...."
1,top250,Блеф (1976),17139,Stasiki,2008-03-04,None,Good,0.0,\nАдриано Челентано продолжает радовать нас св...
2,top250,Блеф (1976),17137,Flashman,2007-03-04,None,Good,10.0,"\nНесомненно, это один из великих фильмов 80-х..."
3,top250,Блеф (1976),17135,Sergio Tishin,2009-08-17,""" Черное, красное, ерунда это все. Выигрывает ...",Good,0.0,\nЭта фраза на мой взгляд отражает сюжет несом...
4,top250,Блеф (1976),17151,Фюльгья,2009-08-20,"«Он хотел убежать? Да! Блеф, блеф…»",Neutral,7.0,"\n- как пела Земфира, скорее всего, по соверше..."


In [4]:
# проверка на сбалансированность
df['grade3'].value_counts(normalize=True)*100

grade3
Good       74.510125
Bad        12.984067
Neutral    12.505807
Name: proportion, dtype: float64

In [5]:
reviews_df = df[['grade3', 'content']]

In [6]:
reviews_df['content'][0]

'\n"Блеф»\xa0—\xa0одна из\xa0моих самых любимых комедий.\n\nЭтот фильм я\xa0наверно смотрел раз\xa0сто, нет\xa0я конечно блефую, я\xa0видел его\xa0куда больше. Не\xa0могу не\xa0выразить своё восхищение главными действующими лицами этого фильма. Начну с\xa0Адриано Челентано для\xa0которого как\xa0я считаю это\xa0лучшая роль в\xa0кино. Великолепный актёр, неплохой певец, странно что\xa0на его\xa0родине в\xa0Италии его\xa0песни мало кто\xa0слушает. Ну\xa0я думаю что\xa0и итальянцы и\xa0французы привыкли к\xa0тому, что\xa0у нас\xa0до сих\xa0их актёры популярней чем\xa0даже на\xa0своей родине. Да, такой вот\xa0парадокс. Челентано конечно профессионал своего дела, комик с\xa0серьёзным выражением лица. Он\xa0смешон ещё\xa0и потому, что\xa0одновременно так\xa0серъёзен. Адриано браво!\n\nА\xa0теперь несколько слов об\xa0Энтони Куине. Да\xa0тот самый горбун из\xa0Нотр-дама. Собор Парижской Богоматери, оригинальная версия, кто\xa0не смотрел рекомендую. С\xa0ним как-то приключилась одна интересная

In [7]:
# ф-я для очистки текста
def clean_text(text):
    all_punct = string.punctuation + '"'
    # удаляем ссылки
    text = re.sub(r'http\S+', " ", text)
    # Сначала разделяем слова по регистру (camelCase и PascalCase)
    text = re.sub(r'([а-яa-z])([А-ЯA-Z])', r'\1 \2', text)
    # перводим в нижний регистр
    text = text.lower()
    # удаляем числа
    text = re.sub(r'\d+', ' ', text)
    # удаляем emoji
    text = re.sub(r'[^\w\s,\.!?;:]', '', text)
    # Удаляем знаки пунктуации, но сохраняем точку, восклицательный и вопросительный знаки
    # для последующей обработки
    punct_to_remove = all_punct.replace('.', '').replace('!', '').replace('?', '')
    text = text.translate(str.maketrans('', '', punct_to_remove))
    # Добавляем пробелы после точек, восклицательных и вопросительных знаков, если их нет
    # Обрабатываем случаи типа "слово.Следующее" -> "слово. Следующее"
    text = re.sub(r'([.!?])([а-яa-z])', r'\1 \2', text)
    # Удаляем оставшиеся знаки пунктуации
    text = text.translate(str.maketrans('', '', all_punct))
     # Заменяем все последовательности пробелов на один пробел
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [8]:
clean_text(reviews_df['content'][0])

'блеф одна из моих самых любимых комедий этот фильм я наверно смотрел раз сто нет я конечно блефую я видел его куда больше не могу не выразить своё восхищение главными действующими лицами этого фильма начну с адриано челентано для которого как я считаю это лучшая роль в кино великолепный актёр неплохой певец странно что на его родине в италии его песни мало кто слушает ну я думаю что и итальянцы и французы привыкли к тому что у нас до сих их актёры популярней чем даже на своей родине да такой вот парадокс челентано конечно профессионал своего дела комик с серьёзным выражением лица он смешон ещё и потому что одновременно так серъёзен адриано браво а теперь несколько слов об энтони куине да тот самый горбун из нотрдама собор парижской богоматери оригинальная версия кто не смотрел рекомендую с ним както приключилась одна интересная история на съёмках одного из своих фильмов он то ли сломал то ли подвихнул ногу а роль требовала от него чтобы в одной из сцен он кружился с дамой в танце и он

In [9]:
# очищенный текст
cleaned_text = []
for text in reviews_df['content']:
  cleaned_text.append(clean_text(text))

### 3. LSTM

In [10]:
# --- Подготовка данных ---
texts = reviews_df['content'].values
labels = reviews_df['grade3'].values

In [11]:
# кодируем метки в числа
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(labels)

In [12]:
# переводим в one-hot (Good/Bad/Neutral → 3 класса)
y = to_categorical(y, num_classes=3)

In [13]:
# Токенизация
max_words = 8000   # количество слов в словаре
max_len = 100       # максимальная длина отзыва

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

X = pad_sequences(sequences, maxlen=max_len)

In [14]:
# --- Разделение ---
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class LSTMClassifier(Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.3, l2_reg=1e-4):
        super(LSTMClassifier, self).__init__()
        
        self.embedding = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        
        # Двунаправленный LSTM + Dropout
        self.lstm = layers.Bidirectional(
            layers.LSTM(hidden_dim, 
                        return_sequences=False, 
                        dropout=dropout, 
                        recurrent_dropout=0.2,
                        kernel_regularizer=tf.keras.regularizers.l2(l2_reg))
        )
        
        self.dropout = layers.Dropout(dropout)
        self.fc = layers.Dense(output_dim, activation="softmax",
                       kernel_regularizer=tf.keras.regularizers.l2(1e-4))
    
    def call(self, x):
        x = self.embedding(x)
        x = self.lstm(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [16]:
# --- Callback для F1-macro ---
class F1MacroCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val, verbose=0)
        y_true = np.argmax(self.y_val, axis=1)
        y_pred_classes = np.argmax(y_pred, axis=1)
        f1 = f1_score(y_true, y_pred_classes, average="macro")
        print(f"\nEpoch {epoch+1} — val_f1_macro: {f1:.4f}")
        logs["val_f1_macro"] = f1

In [17]:
vocab_size = max_words   # max_words
embedding_dim = 32
hidden_dim = 32
output_dim = 3

model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.4)

model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["accuracy"]
)

I0000 00:00:1758887392.966540      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758887392.967251      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [18]:
callbacks = [
    F1MacroCallback(X_val, y_val),
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_lstm.h5", save_best_only=True)
]

In [19]:
from sklearn.utils import class_weight

y_integers = np.argmax(y_train, axis=1)
class_weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_integers),
    y=y_integers
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Weights: {0: 2.567043760413926, 1: 0.44737887819043254, 2: 2.66520986979878}


In [20]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=7,
    batch_size=64,
    callbacks=callbacks,
    class_weight=class_weights
)

Epoch 1/7
458/458 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.7165 - loss: 0.8649
Epoch 1 — val_f1_macro: 0.6610
458/458 ━━━━━━━━━━━━━━━━━━━━ 180s 370ms/step - accuracy: 0.7165 - loss: 0.8646 - val_accuracy: 0.7262 - val_loss: 0.5675 - val_f1_macro: 0.6610
Epoch 2/7
458/458 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.7932 - loss: 0.5151
Epoch 2 — val_f1_macro: 0.6639
458/458 ━━━━━━━━━━━━━━━━━━━━ 166s 363ms/step - accuracy: 0.7932 - loss: 0.5151 - val_accuracy: 0.7426 - val_loss: 0.5727 - val_f1_macro: 0.6639
Epoch 3/7
458/458 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 0.8370 - loss: 0.3955
Epoch 3 — val_f1_macro: 0.6668
458/458 ━━━━━━━━━━━━━━━━━━━━ 166s 363ms/step - accuracy: 0.8370 - loss: 0.3955 - val_accuracy: 0.7593 - val_loss: 0.5839 - val_f1_macro: 0.6668
Epoch 4/7
458/458 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.8687 - loss: 0.3131
Epoch 4 — val_f1_macro: 0.6774
458/458 ━━━━━━━━━━━━━━━━━━━━ 170s 370ms/step - accuracy: 0.8686 - loss: 0.3132 - val_accuracy: 0

In [21]:
from sklearn.metrics import classification_report, confusion_matrix

# предсказания
y_val_pred = model.predict(X_val)
y_val_classes = np.argmax(y_val_pred, axis=1)
y_val_true = np.argmax(y_val, axis=1)

print(classification_report(y_val_true, y_val_classes, target_names=encoder.classes_))
print(confusion_matrix(y_val_true, y_val_classes))

229/229 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step
              precision    recall  f1-score   support

         Bad       0.71      0.69      0.70       950
        Good       0.94      0.85      0.89      5454
     Neutral       0.36      0.56      0.44       915

    accuracy                           0.80      7319
   macro avg       0.67      0.70      0.68      7319
weighted avg       0.84      0.80      0.81      7319

[[ 658   48  244]
 [ 120 4660  674]
 [ 150  253  512]]


In [32]:
import os
# Создаем папку для выходных данных
output_dir = "/kaggle/working/model_package"
os.makedirs(output_dir, exist_ok=True)

In [30]:
from tensorflow.keras.models import load_model

loaded_model = load_model(
    "lstm_reviews.h5",
    custom_objects={"LSTMClassifier": LSTMClassifier}
)

TypeError: Unable to revive model from config. When overriding the `get_config()` method, make sure that the returned config contains all items used as arguments in the  constructor to <class '__main__.LSTMClassifier'>, which is the default behavior. You can override this default behavior by defining a `from_config(cls, config)` class method to specify how to create an instance of LSTMClassifier from its config.

Received config={'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}

Error encountered during deserialization: LSTMClassifier.__init__() got an unexpected keyword argument 'trainable'